In [27]:
import torch 
import numpy as np 
import os 
import pathlib      
import glob   
import torchvision 
from torchvision.transforms import transforms 
from torch.utils.data import DataLoader 
from torch.autograd import Variable
from torch.optim import Adam
import torch.nn as nn
from torch.nn.functional import relu

In [28]:
transformers = transforms.Compose([
    transforms.Resize((150,150)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.5,0.5,0.5],[0.5,0.5,0.5])]
)

In [29]:
batch_size = 256
train_path = "A:\ML\CNN\seg_train\seg_train"#here set the path to u r train dataset
test_path = "A:\ML\CNN\seg_test\seg_test"# here u r test dataset 
train_loader = DataLoader(torchvision.datasets.ImageFolder(train_path,transformers),batch_size = batch_size,shuffle=True)
test_loader = DataLoader(torchvision.datasets.ImageFolder(test_path,transformers),batch_size = batch_size,shuffle=False)

In [30]:
root = pathlib.Path(train_path)
classes = sorted([j.name.split("/")[-1] for j  in root.iterdir()])

In [31]:
print(classes)

['buildings', 'forest', 'glacier', 'mountain', 'sea', 'street']


In [32]:
from torch.nn import Module, Linear, Conv2d, MaxPool2d, BatchNorm2d
from torch.nn.functional import relu
class ConvNet(nn.Module):
    def __init__(self,num_classes=6):
        super(ConvNet,self).__init__()
        
        #Output size after convolution filter
        #((w-f+2P)/s) +1
        
        #Input shape= (256,3,150,150)
        
        self.conv1=nn.Conv2d(in_channels=3,out_channels=12,kernel_size=3,stride=1,padding=1)
        #Shape= (256,12,150,150)
        self.bn1=nn.BatchNorm2d(num_features=12)
        #Shape= (256,12,150,150)
        self.relu1=nn.ReLU()
        #Shape= (256,12,150,150)
        
        self.pool=nn.MaxPool2d(kernel_size=2)
        #Reduce the image size be factor 2
        #Shape= (256,12,75,75)
        
        
        self.conv2=nn.Conv2d(in_channels=12,out_channels=20,kernel_size=3,stride=1,padding=1)
        #Shape= (256,20,75,75)
        self.relu2=nn.ReLU()
        #Shape= (256,20,75,75)
        
        
        
        self.conv3=nn.Conv2d(in_channels=20,out_channels=32,kernel_size=3,stride=1,padding=1)
        #Shape= (256,32,75,75)
        self.bn3=nn.BatchNorm2d(num_features=32)
        #Shape= (256,32,75,75)
        self.relu3=nn.ReLU()
        #Shape= (256,32,75,75)
        
        
        self.fc=nn.Linear(in_features=75 * 75 * 32,out_features=20)
        self.fc1=nn.Linear(in_features=20,out_features=12)
        self.fc2=nn.Linear(in_features=12,out_features=num_classes)
        
        
        
        #Feed forwad function
        
    def forward(self,input):
        output=self.conv1(input)
        output=self.bn1(output)
        output=self.relu1(output)
            
        output=self.pool(output)
            
        output=self.conv2(output)
        output=self.relu2(output)
            
        output=self.conv3(output)
        output=self.bn3(output)
        output=self.relu3(output)
            
            
            #Above output will be in matrix form, with shape (256,32,75,75)
            
        output=output.view(-1,32*75*75)
            
            
        output=self.fc(output)
        output=relu(output)
        output=self.fc1(output)
        output = relu(output)
        output=self.fc2(output)
            
        return output

In [33]:
model = ConvNet(num_classes=6)

In [34]:
optimizer=Adam(model.parameters(),lr=0.001,weight_decay=0.0001)
loss_function=nn.CrossEntropyLoss()

In [35]:
num_epochs=10

In [36]:
train_count=len(glob.glob(train_path+'/**/*.jpg'))
test_count=len(glob.glob(test_path+'/**/*.jpg'))

In [37]:
print(train_count,test_count)

14034 3000


In [38]:
best_accuracy=0.0
for epoch in range(num_epochs):
    model.train()
    train_accuracy=0.0
    train_loss=0.0
    
    for i, (images,labels) in enumerate(train_loader):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())
            
        optimizer.zero_grad()
        outputs=model(images)
        loss=loss_function(outputs,labels)
        loss.backward()
        optimizer.step()
        
        
        train_loss+= loss.cpu().data*images.size(0)
        _,prediction=torch.max(outputs.data,1)
        
        train_accuracy+=int(torch.sum(prediction==labels.data))
        
    train_accuracy=train_accuracy/train_count
    train_loss=train_loss/train_count

     # Evaluation on testing dataset
    model.eval()
    
    test_accuracy=0.0
    for i, (images,labels) in enumerate(test_loader):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())
            
        outputs=model(images)
        _,prediction=torch.max(outputs.data,1)
        test_accuracy+=int(torch.sum(prediction==labels.data))
    
    test_accuracy=test_accuracy/test_count
    
    
    print('Epoch: '+str(epoch)+' Train Loss: '+str(train_loss)+' Train Accuracy: '+str(train_accuracy)+' Test Accuracy: '+str(test_accuracy))
    
    #Save the best model
    if test_accuracy>best_accuracy:
        torch.save(model.state_dict(),'best_generation.model')
        best_accuracy=test_accuracy

Epoch: 0 Train Loss: tensor(2.0308) Train Accuracy: 0.1662391335328488 Test Accuracy: 0.158
Epoch: 1 Train Loss: tensor(1.5880) Train Accuracy: 0.27825281459313095 Test Accuracy: 0.29833333333333334
Epoch: 2 Train Loss: tensor(1.5624) Train Accuracy: 0.31124412141941 Test Accuracy: 0.2926666666666667
Epoch: 3 Train Loss: tensor(1.5432) Train Accuracy: 0.3142368533561351 Test Accuracy: 0.33566666666666667
Epoch: 4 Train Loss: tensor(1.5253) Train Accuracy: 0.3282741912498219 Test Accuracy: 0.33
Epoch: 5 Train Loss: tensor(1.5068) Train Accuracy: 0.33183696736497076 Test Accuracy: 0.33666666666666667


KeyboardInterrupt: 